konlpy 설치
- 환경변수 편집 jvm.dll 경로 확인 : C:\Program Files\Java\jdk1.8.0_261\jre\bin\server


https://data-scientist-brian-kim.tistory.com/79

In [1]:
!pip install konlpy

In [2]:
import konlpy

In [10]:
import pandas as pd

train_df = pd.read_csv('./dataset/ratings_train.txt', sep ='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
train_df.loc[4,:]

id                                                    6483659
document    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
label                                                       1
Name: 4, dtype: object

In [13]:
train_df.loc[4,:].document

'사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'

In [15]:
# 부정 0 -> 75173개, 긍정 1 -> 74827개
train_df.label.value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [11]:
# 데이터 전처리 Cleansing -> Tokenization
import re
# null값있는 경우 공백으로 처리
train_df = train_df.fillna('') 

# sub -> substitute 대체  
# r 이스케이프 문자를 있는 그대로 쓰는 형식
# \d+ 숫자 한개 이상
# x에 대해서 숫자를 공백으로 바꿔주는 lambda 함수
train_df['document'] = train_df['document'].apply(lambda x:re.sub(r"\d+"," ",x))
test_df = pd.read_csv('./dataset/ratings_test.txt',sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x:re.sub(r"\d+"," ",x))

train_df.drop('id', axis=1,inplace=True)
test_df.drop('id',axis=1, inplace=True)

In [12]:
from konlpy.tag import Okt
import konlpy
okt = Okt()

def okt_tokenizer(text):
    # 입력 인자로 들어온 text를 형태소 단어로 토큰화하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

### 텍스트 전처리 순서
- 클렌징 > 토큰화 > 벡터화(CountVectorizer, Term Frequency - Inverse Document Frequency)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings('ignore')

# max_df 빈도수 0 ~ 90%까지의 단어만 피처로 추출 -> 상위 10%는 피처로 추출하지 않는다
# min_df 전체 문서에 걸쳐서 3번 이하로 나타나는 단어는 피처로 추출하지 않는다
# n_gram_range BOW모델의 단어 순서를 어느 정도 보강하기 위한 n_garm 범위 설정 -> 튜플 형태로 (범위 최솟값, 최대값) 지정
tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

C:\Users\HOME\anaconda3\envs\cakd5\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# [과제] lr으로 모델링 및 하이퍼 파라미터 튜닝후 테스트 데이터로 평가하세요.